# This notebook could be used to download and pre-process the ACDC RVIP data and labels and slice them into 2D

In [1]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id

# ------------------------------------------jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# ------------------------------------------ import helpers
# Notebook_imports should import glob, os, and many other standard libs
from src.utils.Notebook_imports import *
# load helper function to slice 3d_volumes into 2d_slices
from src.data.Dataset import ensure_dir, create_2d_slices_from_3d_volume_files
from ipyfilechooser import FileChooser

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/wft21_septum_landmark_detection


# Load CMR and mask file names

In [6]:
# Download and unpack the raw data
# small helper
def clean_import(dir_path):
    import shutil
    try:
        shutil.rmtree(dir_path)
    except OSError as e:
        print("Error: %s : %s" % (dir_path, e.strerror))
        print('Dont worry, irectory will be created.')
    ensure_dir(dir_path)

In [4]:
# Please change only the data_root var
dataroot_path = FileChooser()
dataroot_path.title = '<b>Choose a data root, data will be downloaded and extracted in this dir</b>'
display(dataroot_path)

FileChooser(path='/mnt/ssd/git/wft21_septum_landmark_detection', filename='', title='HTML(value='<b>Choose a d…

In [30]:
path_to_original_acdc_files = FileChooser()
path_to_original_acdc_files.title = '<b>Choose the path to the original "acdc" root-folder with the patient sub-folders</b>'
display(path_to_original_acdc_files)

FileChooser(path='/mnt/ssd/git/wft21_septum_landmark_detection', filename='', title='HTML(value='<b>Choose the…

In [31]:
# define a folder for the acdc cmr and masks, make sure not to use an existing folder
data_root = dataroot_path.selected
import_path = os.path.join(data_root, 'import')
ensure_dir(data_root)
clean_import(import_path)

In [32]:
# download cleaned rvip 3D cmr and masks
!wget https://heibox.uni-heidelberg.de/f/8776d7311ec84723aacf/?dl=1 -P {import_path}
print('downloaded')
# unzip and replace
zip_file = glob.glob(os.path.join(import_path,'index.html?dl=*'))[0]
!unzip -o {zip_file} -d {data_root}
# clean temp import older
clean_import(import_path)

--2021-12-06 15:27:14--  https://heibox.uni-heidelberg.de/f/8776d7311ec84723aacf/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/8ef9102e-2811-4b15-9bff-00657f4786cb/rvips.zip [following]
--2021-12-06 15:27:14--  https://heibox.uni-heidelberg.de/seafhttp/files/8ef9102e-2811-4b15-9bff-00657f4786cb/rvips.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 100323435 (96M) [application/zip]
Saving to: ‘/mnt/ssd/data/temp/import/index.html?dl=1’

index.html?dl=1     100%[===================>]  95,68M  7,64MB/s    in 14s     

2021-12-06 15:27:28 (6,72 MB/s) - ‘/mnt/ssd/data/temp/import/index.html?dl=1’ saved [100323435/100323435]

downloaded
Archive:  /mnt/ssd/data/temp/import/index.h

In [33]:
# remove old and download new cv-dataframe
!rm {data_root}df_kfold.csv -f
!wget https://heibox.uni-heidelberg.de/f/03f57e89dc8b46668144/?dl=1 -P {import_path}
print('downloaded')
# unzip and replace
zip_file = glob.glob(os.path.join(import_path,'index.html?dl=*'))[0]
!unzip -o {zip_file} -d {data_root}
# clean temp import older
clean_import(import_path)

--2021-12-06 15:27:40--  https://heibox.uni-heidelberg.de/f/03f57e89dc8b46668144/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/73de2163-f4a6-4529-b86a-cce6092def46/df_kfold.zip [following]
--2021-12-06 15:27:40--  https://heibox.uni-heidelberg.de/seafhttp/files/73de2163-f4a6-4529-b86a-cce6092def46/df_kfold.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 20549 (20K) [application/zip]
Saving to: ‘/mnt/ssd/data/temp/import/index.html?dl=1’

index.html?dl=1     100%[===================>]  20,07K   115KB/s    in 0,2s    

2021-12-06 15:27:40 (115 KB/s) - ‘/mnt/ssd/data/temp/import/index.html?dl=1’ saved [20549/20549]

downloaded
Archive:  /mnt/ssd/data/temp/import/index.html?dl=

# Download original acdc dataset

In [34]:
!wget https://heibox.uni-heidelberg.de/f/45dc4e1c44754e3c95cb/?dl=1 -P {import_path}
print('downloaded')
# unzip and replace
zip_file = glob.glob(os.path.join(import_path,'index.html?dl=*'))[0]
!unzip -o {zip_file} -d {data_root}
# clean temp import older
clean_import(import_path)

--2021-12-06 15:27:53--  https://heibox.uni-heidelberg.de/f/45dc4e1c44754e3c95cb/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/dde83923-5501-46ab-97d7-210624a4311e/original.zip [following]
--2021-12-06 15:27:53--  https://heibox.uni-heidelberg.de/seafhttp/files/dde83923-5501-46ab-97d7-210624a4311e/original.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 1631346761 (1,5G) [application/zip]
Saving to: ‘/mnt/ssd/data/temp/import/index.html?dl=1’

index.html?dl=1     100%[===================>]   1,52G  5,17MB/s    in 3m 52s  

2021-12-06 15:31:46 (6,70 MB/s) - ‘/mnt/ssd/data/temp/import/index.html?dl=1’ saved [1631346761/1631346761]

downloaded
Archive:  /mnt/ssd/data/temp/impo

# Check the folder structure in data_root

In [36]:
# io == interobserver
# pp == 100 patients x phases xrvip/cmr = 400 files
!tree -L 1 {data_root}

/mnt/ssd/data/temp/
├── 2D
├── df_kfold.csv
├── import
├── io
├── original
├── pp
└── smooth.nii

5 directories, 2 files


# Reuse an existing ACDC directory

--> copy the 3D CMR files from the original ACDC download folder

In [42]:
path_to_acdc_original = path_to_original_acdc_files.selected
print('collect 3D CMR from: {}'.format(path_to_acdc_original))
#searches in all patient folders for any 3D CMR (2 frames per patient) as nifti
images = sorted(glob.glob(os.path.join(path_to_acdc_original, '*/*frame[0-9][0-9].nii.gz')))
print('images: {}'.format(len(images)))

collect 3D CMR from: /mnt/ssd/data/temp/original/
images: 200


# Collect 3D CMR and labels/masks in sorted order

--> make sure both lists are of equal length

In [39]:
# quality check of the image and mask names, find names with wrong names
# give input and output path here
input_path = os.path.join(data_root, 'pp')
export_path = os.path.join(data_root, '2D')
#images = sorted(glob.glob(os.path.join(input_path, '*frame[0-9][0-9].nii'))) 
masks = sorted(glob.glob(os.path.join(input_path, '*frame[0-9][0-9]_rvip.nrrd'))) #searches in all first level folders for any mask as nrrd
print('images: {}'.format(len(images)))
print('masks: {}'.format(len(masks)))
assert(len(images) == len(masks)), 'len(images)-> {} != len(masks)-> {} '.format(len(images), len(masks))
# in the optimal case there should be as many images as masks. If not, some of the annotations might have been saved with a wrong name.

images: 200
masks: 200


In [41]:
# Slice the 3D vol in 2D slices
ensure_dir(export_path)
[create_2d_slices_from_3d_volume_files(img_f=img,mask_f=msk, export_path=export_path) for img,msk in zip(images,masks)]

[['01', [10, 256, 216]],
 ['12', [10, 256, 216]],
 ['01', [10, 256, 232]],
 ['12', [10, 256, 232]],
 ['01', [10, 256, 256]],
 ['15', [10, 256, 256]],
 ['01', [10, 256, 232]],
 ['15', [10, 256, 232]],
 ['01', [10, 216, 256]],
 ['13', [10, 216, 256]],
 ['01', [11, 256, 232]],
 ['16', [11, 256, 232]],
 ['01', [10, 224, 222]],
 ['07', [10, 224, 222]],
 ['01', [10, 256, 200]],
 ['13', [10, 256, 200]],
 ['01', [10, 256, 208]],
 ['13', [10, 256, 208]],
 ['01', [10, 256, 208]],
 ['13', [10, 256, 208]],
 ['01', [9, 256, 216]],
 ['08', [9, 256, 216]],
 ['01', [10, 256, 184]],
 ['13', [10, 256, 184]],
 ['01', [10, 256, 216]],
 ['14', [10, 256, 216]],
 ['01', [10, 216, 256]],
 ['13', [10, 216, 256]],
 ['01', [9, 216, 256]],
 ['10', [9, 216, 256]],
 ['01', [10, 256, 192]],
 ['12', [10, 256, 192]],
 ['01', [9, 256, 216]],
 ['09', [9, 256, 216]],
 ['01', [8, 256, 216]],
 ['10', [8, 256, 216]],
 ['01', [11, 256, 216]],
 ['11', [11, 256, 216]],
 ['01', [8, 256, 208]],
 ['11', [8, 256, 208]],
 ['01', [1